In [1]:
#import packages
import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import torch

In [2]:
#open data
with gzip.open('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))

In [3]:
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

In [175]:
#open cell data
with gzip.open('NCSU-DigIC-GraphData-2023-07-25/cells.json.gz','rb') as f:
    cells = json.loads(f.read().decode('utf-8'))
#cells

In [13]:
#functions for accessing GRCs
def buildBST(array,start=0,finish=-1):
    if finish<0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start==1:
        ltl=start
    else:
        ltl=buildBST(array,start,mid)
    
    if finish-mid==1:
        gtl=mid
    else:
        gtl=buildBST(array,mid,finish)
        
    return((array[mid],ltl,gtl))

def getGRCIndex(x,y,xbst,ybst):
    while (type(xbst)==tuple):
        if x < xbst[0]:
            xbst=xbst[1]
        else:
            xbst=xbst[2]
            
    while (type(ybst)==tuple):
        if y < ybst[0]:
            ybst=ybst[1]
        else:
            ybst=ybst[2]
            
    return ybst, xbst

In [21]:
#open congestion data
congestion_data = np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])
capacity = np.zeros(shape = [instances.shape[0],])
indices = []

In [22]:
#extract congestion data from GRCs and apply values to individual cell instances
for k in range(instances.shape[0]):
    #print(k)
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    c = 0
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
        c += congestion_data['capacity'][lyr][i][j]
    demand[k] = d
    capacity[k] = c
    indices.append((i, j))
        
instances['routing_demand'] = demand
instances['routing_capacity'] = capacity
instances['grc_index'] = indices

In [23]:
instances

,name,id,xloc,yloc,cell,orient,routing_demand,routing_capacity,grc_index
0,clk_gate_out_reg/latch,0,41984,44544,23,0,20.0,28.0,"(29, 27)"
1,clk_gate_out_reg_0/latch,1,41984,47616,23,6,23.0,28.0,"(31, 27)"
2,clk_gate_out_reg_1/latch,2,44160,44544,23,0,23.0,33.0,"(29, 28)"
3,clk_gate_out_reg_2/latch,3,44160,47616,23,0,22.0,33.0,"(31, 28)"
4,clk_gate_out_reg_3/latch,4,46336,47616,23,0,21.0,23.0,"(31, 29)"
...,...,...,...,...,...,...,...,...,...
3947,U4123,3947,21888,53760,42,4,31.0,31.0,"(35, 13)"
3948,U4125,3948,33664,66048,42,0,30.0,30.0,"(43, 21)"
3949,U4128,3949,23296,66048,34,0,27.0,29.0,"(43, 14)"
3950,ZCTSBUF_205_132,3950,40576,44544,11,0,28.0,33.0,"(29, 26)"


In [32]:
#extract pin counts and cell sizes from cells dictionary
individual_pins = []
cell_widths = []
cell_heights = []
for k in range(instances.shape[0]):
    cell_type = instances.iloc[k]['cell']
    cell_pins = len(cells[cell_type]['terms'])
    cell_width = cells[cell_type]['width']
    cell_height = cells[cell_type]['height']
    individual_pins.append(cell_pins)
    cell_widths.append(cell_width)
    cell_heights.append(cell_height)

instances['individual_pins'] = individual_pins
instances['width'] = cell_widths
instances['height'] = cell_heights
instances

,name,id,xloc,yloc,cell,orient,routing_demand,routing_capacity,grc_index,individual_pins,grc_pin_count,grc_cell_count,width,height
0,clk_gate_out_reg/latch,0,41984,44544,23,0,20.0,28.0,"(29, 27)",4,12,3,2176,1536
1,clk_gate_out_reg_0/latch,1,41984,47616,23,6,23.0,28.0,"(31, 27)",4,8,2,2176,1536
2,clk_gate_out_reg_1/latch,2,44160,44544,23,0,23.0,33.0,"(29, 28)",4,10,3,2176,1536
3,clk_gate_out_reg_2/latch,3,44160,47616,23,0,22.0,33.0,"(31, 28)",4,8,2,2176,1536
4,clk_gate_out_reg_3/latch,4,46336,47616,23,0,21.0,23.0,"(31, 29)",4,4,1,2176,1536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,U4123,3947,21888,53760,42,4,31.0,31.0,"(35, 13)",3,12,3,512,1536
3948,U4125,3948,33664,66048,42,0,30.0,30.0,"(43, 21)",3,16,4,512,1536
3949,U4128,3949,23296,66048,34,0,27.0,29.0,"(43, 14)",2,17,6,384,1536
3950,ZCTSBUF_205_132,3950,40576,44544,11,0,28.0,33.0,"(29, 26)",2,12,3,640,1536


In [30]:
#extract the number of pins in each grc
instances['grc_pin_count'] = instances['individual_pins'].groupby(instances['grc_index']).transform('sum')
instances['grc_pin_count'].value_counts()

grc_pin_count
20    734
15    704
19    378
10    316
12    249
14    226
17    215
18    215
16    214
13    100
6      89
22     85
24     82
11     79
21     79
9      56
5      51
25     30
23     21
8      10
3       7
26      5
7       4
4       3
Name: count, dtype: int64

In [29]:
#extract the number of pins in each grc
instances['grc_cell_count'] = instances['individual_pins'].groupby(instances['grc_index']).transform('count')
instances['grc_cell_count'].value_counts()

grc_cell_count
4    1644
3    1131
2     550
5     430
1     142
6      48
7       7
Name: count, dtype: int64

In [33]:
#create feature matrix by dropping irrelevant features
feature_matrix = instances.drop(columns=['name', 'id', 'cell', 'orient', 'routing_demand', 'routing_capacity', 'grc_index'])
feature_matrix

,xloc,yloc,individual_pins,grc_pin_count,grc_cell_count,width,height
0,41984,44544,4,12,3,2176,1536
1,41984,47616,4,8,2,2176,1536
2,44160,44544,4,10,3,2176,1536
3,44160,47616,4,8,2,2176,1536
4,46336,47616,4,4,1,2176,1536
...,...,...,...,...,...,...,...
3947,21888,53760,3,12,3,512,1536
3948,33664,66048,3,16,4,512,1536
3949,23296,66048,2,17,6,384,1536
3950,40576,44544,2,12,3,640,1536


In [36]:
#create labels with demand
labels = instances['routing_demand']
labels

0       20.0
1       23.0
2       23.0
3       22.0
4       21.0
        ... 
3947    31.0
3948    30.0
3949    27.0
3950    28.0
3951    23.0
Name: routing_demand, Length: 3952, dtype: float64

In [108]:
#open connectivity data
conn=np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_connectivity.npz')
#A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
#A

In [109]:
import torch

In [117]:
#create edge-index of data
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

A_coo = A.tocoo()
edge_index = np.array([A_coo.row, A_coo.col])
edge_index = torch.tensor(edge_index, dtype=torch.int64)

In [112]:
instances

,name,id,xloc,yloc,cell,orient,routing_demand,routing_capacity,grc_index,individual_pins,grc_pin_count,grc_cell_count,width,height
0,clk_gate_out_reg/latch,0,41984,44544,23,0,20.0,28.0,"(29, 27)",4,12,3,2176,1536
1,clk_gate_out_reg_0/latch,1,41984,47616,23,6,23.0,28.0,"(31, 27)",4,8,2,2176,1536
2,clk_gate_out_reg_1/latch,2,44160,44544,23,0,23.0,33.0,"(29, 28)",4,10,3,2176,1536
3,clk_gate_out_reg_2/latch,3,44160,47616,23,0,22.0,33.0,"(31, 28)",4,8,2,2176,1536
4,clk_gate_out_reg_3/latch,4,46336,47616,23,0,21.0,23.0,"(31, 29)",4,4,1,2176,1536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3947,U4123,3947,21888,53760,42,4,31.0,31.0,"(35, 13)",3,12,3,512,1536
3948,U4125,3948,33664,66048,42,0,30.0,30.0,"(43, 21)",3,16,4,512,1536
3949,U4128,3949,23296,66048,34,0,27.0,29.0,"(43, 14)",2,17,6,384,1536
3950,ZCTSBUF_205_132,3950,40576,44544,11,0,28.0,33.0,"(29, 26)",2,12,3,640,1536


In [146]:
import torch.nn.functional as F
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv

class GAT(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out, heads=8):
        super().__init__()
        self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
        #arbitrarily set output of second layer to have the same number of dimensions as dim_in
        self.gat2 = GATv2Conv(dim_h*heads, dim_in, heads=1)
        self.out = Linear(dim_in, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.6, training=self.training)
        h = self.gat1(x, edge_index)
        h = F.elu(h)
        h = F.dropout(h, p=0.6, training=self.training)
        h = self.gat2(h, edge_index)
        h = F.elu(h)
        h = self.out(h)
        return h, F.log_softmax(h, dim=1)

def accuracy(pred_y, y):
    return ((pred_y == y).sum() / len(y)).item()

def train(model, x, y, edge_index, train_mask, val_mask):
    criterion = torch.nn.MSELoss()
    optimizer = model.optimizer
    epochs = 200

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        _, out = model(x, edge_index)
        loss = criterion(out[train_mask], y[train_mask])
        #acc = accuracy(out[train_mask].argmax(dim=1), y[train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        val_loss = criterion(out[val_mask], y[val_mask])
        val_acc = accuracy(out[val_mask].argmax(dim=1), y[val_mask])

        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  f'{acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                  f'Val Acc: {val_acc*100:.2f}%')
          
    return model

def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

In [126]:
#move features into tensor
#x = instances.drop(columns=['name', 'id', 'routing_demand'])
x = torch.tensor(feature_matrix.values, dtype=torch.float)

#move target into tensor
y = torch.tensor(instances['routing_demand'].values, dtype=torch.float)

#create train and validation masks
train_mask = (np.zeros(feature_matrix.shape[0]))
for i in range(2000):
    train_mask[i] += 1
train_mask = torch.tensor(train_mask.astype(bool))
train_mask

val_mask = (np.zeros(feature_matrix.shape[0]))
for i in range(2000, 3000):
    val_mask[i] += 1
val_mask = torch.tensor(val_mask.astype(bool))
val_mask

tensor([False, False, False,  ..., False, False, False])

In [170]:
import torch.nn.functional as F
from torch.nn import Linear, Dropout
from torch_geometric.nn import GCNConv, GATv2Conv

class GAT(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out, heads=8):
        super().__init__()
        self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
        #arbitrarily set output of second layer to have the same number of dimensions as dim_in
        self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)
        #self.out = Linear(dim_in, dim_out)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)

    def forward(self, x, edge_index):
        #h = F.dropout(x, p=0.6, training=self.training)
        h = self.gat1(x, edge_index)
        h = F.elu(h)
        #h = F.dropout(h, p=0.6, training=self.training)
        h = self.gat2(h, edge_index)
        #h = F.elu(h)
        #h = self.out(h)
        return h

#def accuracy(pred_y, y):
#    return ((pred_y == y).sum() / len(y)).item()

def train(model, x, y, edge_index):
    criterion = torch.nn.MSELoss()
    optimizer = model.optimizer
    epochs = 1000

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        out = model(x, edge_index)
        loss = criterion(out, y)
        #acc = accuracy(out[train_mask].argmax(dim=1), y[train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        #val_loss = criterion(out[val_mask], y[val_mask])
        #val_acc = accuracy(out[val_mask].argmax(dim=1), y[val_mask])

        # Print metrics every 10 epochs
        if(epoch % 100 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  )
          
    return model, out

def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

In [172]:
%%time


# Create GAT
gat = GAT(feature_matrix.shape[1], 8, 1)
print(gat)

# Train
model, out = train(gat, x, y, edge_index)#, train_mask, val_mask)
out

# Test
#acc = test(gat, data)
#print(f'GAT test accuracy: {acc*100:.2f}%\n')
#view raw

GAT(
  (gat1): GATv2Conv(7, 8, heads=8)
  (gat2): GATv2Conv(64, 1, heads=1)
)
Epoch   0 | Train Loss: 93884344.000 | Train Acc: 
Epoch 100 | Train Loss: 292880.688 | Train Acc: 
Epoch 200 | Train Loss: 90786.953 | Train Acc: 
Epoch 300 | Train Loss: 46118.133 | Train Acc: 
Epoch 400 | Train Loss: 27698.166 | Train Acc: 
Epoch 500 | Train Loss: 12201.369 | Train Acc: 
Epoch 600 | Train Loss: 9208.413 | Train Acc: 
Epoch 700 | Train Loss: 7206.833 | Train Acc: 
Epoch 800 | Train Loss: 6939.729 | Train Acc: 
Epoch 900 | Train Loss: 9519.888 | Train Acc: 
Epoch 1000 | Train Loss: 3885.096 | Train Acc: 
CPU times: user 16min 10s, sys: 5min 14s, total: 21min 24s
Wall time: 5min 8s


tensor([[-90.2926],
        [-34.8040],
        [-34.8047],
        ...,
        [-20.3298],
        [-35.6850],
        [-34.8045]], grad_fn=<AddBackward0>)